# Story and Review Generation using Langchain

### Importing Necessary Dependencies

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import os
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, Cohere
from langchain.schema import HumanMessage, SystemMessage, AIMessage

### Defining API keys as environment variables

**Where to get the API keys from?**

`OpenAI API Key` - https://platform.openai.com/account/api-keys

`Cohere API Key` - https://dashboard.cohere.com/api-keys

In [2]:
os.environ["OPENAI_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
os.environ["COHERE_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

### Creating two model instances one each for story and review generation

This is where I happened to struggle a bit. I will explain how I have went through this phase which took most of my time (Around 75%).

- After using `OpenAI model` for `story generation` which is `highly reliable` model with comparitively high input/output token limit, I **couldn't find any equally reliable model** for consistent and reliable output delivery **for review generation**.

- I have gone through most important models available in HuggingFaceHub model repositories and tried my hands over many models using inference API of HuggingFaceHub.

- Got access in a very short span of time in meta waitlist and tried the latest `Llama2-70b` model but it isn't providing API access as of now.

- Then tried with `google-fang` models in HuggingFaceHub model space. Though they were accessible easily, they didn't provide output for a set no. of tokens.

- Then searched whether there's access for Google's `BARD API` and Google's `PALM` model. But BARD doesn't provide API services as of now and PALM API available only in US and that too with quite a big waitlist to get access... So, left this option as well.

- Then randomly I went through an old twitter(now X) DM from a developer where he seem to have asked me to test his new library named `litellm` that he built. But, I haven't noticed it back then and now went through his repo and found that he used `Cohere` somewhere. That's when I tried my hands with the same, got API access and have luckily found it to be `integrated with Langchain` as well.

Also **important thing to notice** here is **even with cohere's nightly models, I couldn't get reliable results** for review generation though it was way better than other models that I have tried. Then I have gone through cohere website completely and found a way to build a **custom model with cohere as base model.** Then I `created custom jsonl dataset` with some `40 examples` of how the prompt would be and how the response should be using GPT 3.5 Turbo and used the same to fine-tune the custom model. That's what I have used below as cohere model. You can see that I have given a lengthy string for model parameter - `'a6cbb8d1-1bb6-415c-9543-c0484ee3a5b8-ft'`

In [3]:
story_llm = ChatOpenAI(temperature = 0.9)
review_llm = Cohere(model = 'a6cbb8d1-1bb6-415c-9543-c0484ee3a5b8-ft', temperature = 0.25)

### Story Generation

Here I have used some nuanced way of providing prompt to the LLM using Langchain's extraordinary methods.

In [4]:
message = [
            SystemMessage(content = "You are a nice AI bot named Lee that children all over the world love a lot as you help them sleep at night daily with a short and sweet story in the genre they ask for"),
            HumanMessage(content = "Hey Lee, today I would like to hear a story from you in {} genre. Can you help me sleep today?"),
            AIMessage(content = "Hey kiddo, that sounds like a nice genre to go for and the surprise is, I too have a story with me in that genre. Shall we start our story today?"),
            HumanMessage(content = "Yeah please go ahead with our story Lee")
            ]

In [5]:
#  To dynamically change the genre we have done something over here...

genre = 'sci-fi'

message[1].content = message[1].content.replace('{}', genre)

In [6]:
convo = story_llm(message)

story = convo.content

print(story)

Once upon a time in a distant galaxy, there was a young girl named Luna. Luna lived on a small planet called Nova, which was known for its vibrant and breathtaking celestial displays.

Luna was a curious and adventurous child, always yearning to explore the mysteries of the universe. She spent her days studying the stars and dreaming of far-off galaxies. One night, as she gazed at the sky, she noticed a streak of light shooting across the horizon.

Intrigued by this unusual occurrence, Luna followed the path of the shooting star. To her astonishment, she discovered a crashed spaceship hidden among the trees. She cautiously approached the ship, her heart racing with excitement.

Inside, she found a small, friendly extraterrestrial creature named Zephyr. Zephyr was from a distant planet called Nebula, and he had crash-landed on Nova while on a mission to explore the cosmos.

Luna and Zephyr quickly became friends. Zephyr shared stories of his adventures across galaxies, dazzling Luna wit

### Review Generation

Here the prompt template below might look so simple. But, that was made possible only because of the custom model that we built using Cohere.

In [7]:
prompt_template = """{story}
                     Write a review on above story in 3 sentences.
                  """

prompt =  PromptTemplate(input_variables=['story'], template=prompt_template)

In [8]:
# Filling the placeholder with appropriate variable i.e; story

PROMPT = prompt.format(story = story).strip()

In [9]:
review = review_llm.generate([PROMPT]).generations[0][0].text
print(f'Review: {review}')

Review:  The story of Luna and Zephyr is a captivating blend of friendship, adventure, and the wonders of the universe. The narrative transports young readers to a world where space exploration is a dream and where friendship across galaxies is a reality. Through its vivid portrayal of Luna's adventures and the bond she forms with Zephyr, the story resonates as a reminder that the cosmos is calling, and the best adventures are those shared.


### Structurising the Output

Now that we have understood the process, let's compile everything into a single function and call it to get both the story and review.

In [10]:
def generate_story_and_review(openai_api_key, cohere_api_key, genre = 'sci-fi'):
    
    """This function will generate a story in the genre of your interest using OpenAI's chat model and 
        reviews the same story using custom model made by me using CohereAI
    Args:
        genre (str, optional): Genre of your interest in which you want story. Defaults to 'science-fiction'
        openai_api_key (str): Provide the API key of OpenAI model. Get it from here - https://platform.openai.com/account/api-keys
        cohere_api_key (str): Provide the API key of Cohere model. Get it from here - https://dashboard.cohere.com/api-keys
    Raises:
        TypeError: If parameters not of desired datatype
        Exception: Any other error
    Returns:
        str: Story and Review as two seperate strings
    """
    
    try:
        
        if type(openai_api_key) != str or type(cohere_api_key) != str:
            raise TypeError("API key must be of string datatype. Please check whether you have provided string input in both cases")
        else:
            os.environ["OPENAI_API_KEY"] = openai_api_key
            os.environ["COHERE_API_KEY"] = cohere_api_key
            
            story_llm = ChatOpenAI(temperature = 0.9)
            review_llm = Cohere(model = 'a6cbb8d1-1bb6-415c-9543-c0484ee3a5b8-ft', temperature = 0.25)
            
            message = [
                        SystemMessage(content = "You are a nice AI bot named Lee that children all over the world love a lot as you help them sleep at night daily with a short and sweet story in the genre they ask for"),
                        HumanMessage(content = "Hey Lee, today I would like to hear a story from you in {} genre. Can you help me sleep today?"),
                        AIMessage(content = "Hey kiddo, that sounds like a nice genre to go for and the surprise is, I too have a story with me in that genre. Shall we start our story today?"),
                        HumanMessage(content = "Yeah please go ahead with our story Lee")
                        ]
        
            if type(genre) != str:
                raise TypeError("Genre must be of string datatype. Please check whether you have provided input of correct datatype")
            else:
                message[1].content = message[1].content.replace('{}', genre)
                story = story_llm(message).content
                
                prompt_template = """{story}
                                     Write a review on above story in 3 sentences.
                                  """
                prompt =  PromptTemplate(input_variables=['story'], template = prompt_template)
                PROMPT = prompt.format(story = story).strip()
                
                review = review_llm.generate([PROMPT]).generations[0][0].text
        
        return story, review
        
    except Exception as e:
        print(f"An error occurred: {e}")

In [11]:
story, review = generate_story_and_review(
                                          openai_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                                          cohere_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                                          genre = "fantasy"
                                         )

In [12]:
print(f"Story:\n\n{story}", end = "\n"*4)
print(f"Review:\n\n{review}")

Story:

Once upon a time, in a land far, far away, there was a little girl named Lily. Lily lived in a small village surrounded by thick, enchanted forests. This village was full of ordinary people, but Lily was no ordinary girl. She had a magical gift - the ability to talk to animals.

One day, while exploring the woods, Lily stumbled upon a wounded baby dragon. The dragon had a twisted wing and was unable to fly. Lily's heart filled with compassion, and she decided to help the dragon. She gently approached the dragon, speaking softly and reassuringly.

To her surprise, the dragon understood her words and responded with a low, rumbling purr. Lily carefully tended to the dragon's wing, placing a splint and bandages to help it heal. Days turned into weeks, and Lily visited the dragon every day, bringing it food and keeping it company.

As the dragon's wing began to heal, Lily realized that she needed to find a way to teach the dragon to fly. She consulted the wise old owl who lived in t

<br>
<br>
We could have replicated the same in a streamlit app. But, as there are not many inputs to get from users there won't be much to display over there... That's why I haven't touched that part.
<br>
<br>
Thanks for the opportunity 🙏🏻...
<br>
<br>

Developer Details:

[LinkedIn](https://www.linkedin.com/in/nirmal-data-scientist/)

[GitHub](https://github.com/nirmals-workspace)

[Medium](nirmal-ai.medium.com)

[Updated Resume](https://drive.google.com/file/d/1O5cOW-GxIxJo32wR_ibtGnGJJ2JHmY4z/view?usp=sharing)